<a href="https://colab.research.google.com/github/finardi/Data-Engenieer/blob/main/MovieLens_NM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [24]:
# basics
import os
import gc
import math
import pickle
import numpy as np
import pandas as pd
from time import time

# scipy
import scipy.sparse as sp

# plots
import seaborn as sns
import matplotlib.pyplot as plt

# torch
import torch
from torch.utils.data import Dataset, DataLoader

# retrieval topK
import heapq  

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
manual_seed = 2357 # only primers ;)
 
def deterministic(rep=True):
    if rep:
        np.random.seed(manual_seed)
        torch.manual_seed(manual_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(manual_seed)
            torch.cuda.manual_seed_all(manual_seed)
        torch.backends.cudnn.enabled = False 
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        print(f'Experimento deterministico, seed: {manual_seed} -- ', end = '')
        print(f'Existe {torch.cuda.device_count()} GPU\
 {torch.cuda.get_device_name(0)} disponível.')
    else:
        print('Experimento randomico')
deterministic()

Experimento deterministico, seed: 2357 -- Existe 1 GPU Tesla P100-PCIE-16GB disponível.


# Data prep

In [3]:
!unzip /content/drive/MyDrive/Colab\ Notebooks/RecSys/movielens_datasets/ml100k/ml-100k.zip
!mkdir Data

INPUT_PATH = 'ml-100k/u.data'

OUTPUT_PATH_TRAIN = 'Data/movielens.train.rating'
OUTPUT_PATH_TEST = 'Data/movielens.test.rating'
USER_FIELD = 'userID'

Archive:  /content/drive/MyDrive/Colab Notebooks/RecSys/movielens_datasets/ml100k/ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [4]:
def save_to_csv(df,path, header=False, index=False, sep='\t', verbose=False):
    if verbose:
        print("Saving df to path: {}".format(path))
        print("Columns in df are: {}".format(df.columns.tolist()))

    df.to_csv(path, header=header, index=index, sep=sep)

def get_train_test_df(transactions):
    print("Size of the entire dataset:{}".format(transactions.shape))
    transactions.sort_values(by=['timestamp'], inplace=True)
    last_transaction_mask = transactions.duplicated(subset={USER_FIELD}, keep="last")
    train_df = transactions[last_transaction_mask]
    test_df = transactions[~last_transaction_mask]
    
    train_df.sort_values(by=["userID", 'timestamp'], inplace=True)
    test_df.sort_values(by=["userID", 'timestamp'], inplace=True)
    return train_df, test_df

def report_stats(transactions, train_df, test_df):
    whole_size = transactions.shape[0]*1.0
    train_size = train_df.shape[0]
    test_size = test_df.shape[0]
    print("Total No. of Records = {}".format(whole_size))
    print("Train size = {}, Test size = {}".format(train_size, test_size))
    print("Train % = {}, Test % ={}".format(train_size/whole_size, test_size/whole_size))

transactions = pd.read_csv(
    INPUT_PATH, 
    sep="\t", 
    names=['userID', 'movieID', 'rating', 'timestamp'], 
    engine='python'
    )

transactions['rating'] = 1

# --> make the dataset
train_df, test_df = get_train_test_df(transactions)
save_to_csv(train_df, OUTPUT_PATH_TRAIN, header=False, index=False, verbose=1)
save_to_csv(test_df, OUTPUT_PATH_TEST, header=False, index=False, verbose=1)
report_stats(transactions, train_df, test_df)

Size of the entire dataset:(100000, 4)
Saving df to path: Data/movielens.train.rating
Columns in df are: ['userID', 'movieID', 'rating', 'timestamp']
Saving df to path: Data/movielens.test.rating
Columns in df are: ['userID', 'movieID', 'rating', 'timestamp']
Total No. of Records = 100000.0
Train size = 99057, Test size = 943
Train % = 0.99057, Test % =0.00943


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


# Dataset

In [109]:
class RecSysDataset(Dataset):
    def __init__(self, file_name, num_negatives_train=5, num_negatives_test=100):

        self.trainMatrix = self.load_rating_file_as_matrix(file_name + ".train.rating")
        self.num_users, self.num_items = self.trainMatrix.shape
        self.user_input, self.item_input, self.ratings = self.get_train_instances(self.trainMatrix, num_negatives_train)
        self.testRatings = self.load_rating_file_as_list(file_name + ".test.rating")
        self.testNegatives = self.create_negative_file(num_samples=num_negatives_test)
        
        assert len(self.testRatings) == len(self.testNegatives)

    def get_train_instances(self, train, num_negatives):
        user_input, item_input, ratings = [], [], []
        num_users, num_items = train.shape
        for (u, i) in train.keys():
            # positive instance
            user_input.append(u)
            item_input.append(i)
            ratings.append(1)
            # negative instances
            for _ in range(num_negatives):
                j = np.random.randint(1, num_items)
                # while train.has_key((u, j)):
                while (u, j) in train:
                    j = np.random.randint(1, num_items)
                user_input.append(u)
                item_input.append(j)
                ratings.append(0)
        return user_input, item_input, ratings

    def load_rating_file_as_list(self, filename):
        ratingList = []
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item = int(arr[0]), int(arr[1])
                ratingList.append([user, item])
                line = f.readline()
        return ratingList

    def create_negative_file(self, num_samples=100):
        negativeList = []
        for user_item_pair in self.testRatings:
            user = user_item_pair[0]
            item = user_item_pair[1]
            negatives = []
            for t in range(num_samples):
                j = np.random.randint(1, self.num_items)
                while (user, j) in self.trainMatrix or j == item:
                    j = np.random.randint(1, self.num_items)
                negatives.append(j)
            negativeList.append(negatives)
        return negativeList

    def load_rating_file_as_matrix(self, filename):
        num_users, num_items = 0, 0
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                u, i = int(arr[0]), int(arr[1])
                num_users = max(num_users, u)
                num_items = max(num_items, i)
                line = f.readline()
        
        mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
        with open(filename, "r") as f:
            line = f.readline()
            while line != None and line != "":
                arr = line.split("\t")
                user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
                if (rating > 0):
                    mat[user, item] = 1.0
                line = f.readline()
        return mat
        
    def __len__(self):
        return len(self.user_input)

    def __getitem__(self, index):
        user_id = self.user_input[index]
        item_id = self.item_input[index]
        rating = self.ratings[index]

        return {'user_id': user_id, 'item_id': item_id, 'rating': rating}

# --> Testing RecSysDataset

ds = RecSysDataset(
    file_name='Data/movielens', 
    num_negatives_train=4,
    num_negatives_test=100
    )
    
ds_dict = ds[0]
print('Testing RecSysDataset')
print(f"user_id: {ds_dict['user_id']}, item_id: {ds_dict['item_id']}, rating: {ds_dict['rating']}")

train, testRatings, testNegatives = ds.trainMatrix, ds.testRatings, ds.testNegatives
num_users, num_items = train.shape

print(f"train: {train.shape}, testRatings: {len(testRatings)}, testNegatives: {len(testNegatives)}")
print(f"num_users: {num_users}, num_items: {num_items}")

Testing RecSysDataset
user_id: 1, item_id: 168, rating: 1
train: (944, 1683), testRatings: 943, testNegatives: 943
num_users: 944, num_items: 1683


# Dataloader

In [113]:
BATCH_SZ = 256

train_loader = DataLoader(
    ds, 
    batch_size=BATCH_SZ,
    shuffle=True, 
    num_workers=os.cpu_count(), 
    pin_memory=True
    )

# --> Testing Dataloader
dict_loader = next(iter(train_loader))
print(f"user_id: {dict_loader['user_id'].shape}, item_id: {dict_loader['item_id'].shape}, rating: {dict_loader['rating'].shape}")

print('len dataloader:', len(train_loader))

user_id: torch.Size([256]), item_id: torch.Size([256]), rating: torch.Size([256])
len dataloader: 1935


# Train and Evaluate functions

# Model

In [134]:
class RecSysNet(torch.nn.Module):
    def __init__(self, n_users, n_items,  embedding_dim=16, out=1):
        super().__init__()

        self.activation = torch.nn.ReLU()
        self.drop = torch.nn.Dropout()
        
        embedding_dim = embedding_dim

        self.user_embedding = torch.nn.Embedding(n_users, embedding_dim//2)
        self.item_embedding = torch.nn.Embedding(n_items, embedding_dim//2)

        self.fc_layer_0 = torch.nn.Linear(embedding_dim, 2 * embedding_dim)
        self.ln_0 = torch.nn.LayerNorm(2 * embedding_dim)

        self.fc_layer_1 = torch.nn.Linear(2 * embedding_dim, embedding_dim)
        self.ln_1 = torch.nn.LayerNorm(embedding_dim)

        self.fc_layer_2 = torch.nn.Linear(embedding_dim, embedding_dim//2)
        self.ln_2 = torch.nn.LayerNorm(embedding_dim//2)

        self.output_layer = torch.nn.Linear(embedding_dim//2, out)

    def forward(self, feed_dict):
        user_embedding = self.user_embedding(feed_dict['user_id'])
        item_embedding = self.item_embedding(feed_dict['item_id'])

        x = torch.cat([user_embedding, item_embedding], dim=1)

        # o = self.drop(self.activation(self.fc_layer_0(x)))

        o = self.activation(self.ln_0(self.fc_layer_0(x)))
        o = self.activation(self.ln_1(self.fc_layer_1(o)))
        o = self.activation(self.ln_2(self.fc_layer_2(o)))
        
        logit = self.output_layer(o)
        rating = torch.sigmoid(logit)
        return rating

    def predict(self, feed_dict):
        for key in feed_dict:
            if type(feed_dict[key]) != type(None):
                feed_dict[key] = torch.from_numpy(feed_dict[key]).to(dtype=torch.long, device=device)
        output_scores = self.forward(feed_dict)
        return output_scores.cpu().detach().numpy()

# --> Testing the Network
try:
    del model
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

model = RecSysNet(
    n_users=num_users, 
    n_items=num_items, 
)    

with torch.no_grad():
    out = model(dict_loader)
print(f'out forward shape: {out.shape}', end= '  --  ')
print(f'out sample: {out[0]}')

out forward shape: torch.Size([256, 1])  --  out sample: tensor([0.6071])


In [135]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
_ = 49 

print('\n','#' * _, f'\n # Número de params. {count_parameters(model):,}' \
       ' trainable parameters #\n', '#' * _,'\n')  
model


 ################################################# 
 # Número de params. 22,345 trainable parameters #
 ################################################# 



RecSysNet(
  (activation): ReLU()
  (drop): Dropout(p=0.5, inplace=False)
  (user_embedding): Embedding(944, 8)
  (item_embedding): Embedding(1683, 8)
  (fc_layer_0): Linear(in_features=16, out_features=32, bias=True)
  (ln_0): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (fc_layer_1): Linear(in_features=32, out_features=16, bias=True)
  (ln_1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (fc_layer_2): Linear(in_features=16, out_features=8, bias=True)
  (ln_2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
  (output_layer): Linear(in_features=8, out_features=1, bias=True)
)

In [151]:
def train_one_epoch(model, data_loader, loss_fn, optimizer, epoch_no, num_epoch, device, verbose=True):
    print(f"Epoch [{epoch_no}/{num_epoch}]: ", end= '')
    epoch_loss = []
    model.train()
    for dict_train in data_loader:
        optimizer.zero_grad()
        
        for key in dict_train:
            if type(dict_train[key]) != type(None):
                dict_train[key] = dict_train[key].to(dtype=torch.long, device=device)

        rating_prediction = model(dict_train)

        rating_true = dict_train['rating']
        rating_true = rating_true.float().view(rating_prediction.size())  
        loss = loss_fn(rating_prediction, rating_true)

        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())

    epoch_loss = np.mean(epoch_loss)
    if verbose:
        print(f"Train Loss: {epoch_loss:.4}")
    return epoch_loss
        
def test(model, full_dataset: RecSysDataset, testRatings, topK):
    model.eval()
    hits, ndcgs, mrrs = evaluate_model(model, full_dataset, testRatings, topK)
    hr = np.array(hits).mean()
    ndcg = np.array(ndcgs).mean()
    mrr = np.array(mrrs).mean()

    print(f'              Eval: MRR = {mrr:.4} -- HR = {hr:.4} -- NDCG = {ndcg:.4}')
    return hr, ndcg, mrr
    
def get_items_interacted(user_id, interaction_df):
    userid_mask = interaction_df['userid'] == user_id
    interacted_items = interaction_df.loc[userid_mask].courseid
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

def evaluate_model(model, dataset: RecSysDataset, testRatings: list, topK: int):
    testRatings = dataset.testRatings
    testNegatives = dataset.testNegatives

    mrrs, hits, ndcgs = [], [], []
    for idx in range(len(testRatings)):
        hr, ndcg, mrr = _eval_one_rating(
            idx, 
            model, 
            dataset, 
            testRatings[idx], 
            testNegatives[idx],
            topK
            )
        hits.append(hr)
        ndcgs.append(ndcg)
        mrrs.append(mrr)
    return hits, ndcgs, mrrs

def _eval_one_rating(idx, model, dataset, testRatings, testNegatives, topk=10):
    rating = testRatings
    items = testNegatives
    u = rating[0]

    gtItem = rating[1]
    items.append(gtItem)
    map_item_score = {}
    users = np.full(len(items), u, dtype='int32')

    feed_dict = {
        'user_id': users,
        'item_id': np.array(items),
    }
    
    predictions = model.predict(feed_dict)
    
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]

    ranklist = heapq.nlargest(topk, map_item_score, key=map_item_score.get)
    
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    mrr = getMRR(ranklist, gtItem)
    
    return hr, ndcg, mrr

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0 

def getMRR(ranklist, gtItem):
    for i, pred in enumerate(ranklist):
        if pred == gtItem:
            return 1/(i+1)
    else:
        return 0       

def plot_statistics(hr_list, ndcg_list, loss_list, path):
    plt.figure()
    hr = np.array(hr_list)
    ndcg = np.array(ndcg_list)
    loss = np.array(loss_list)
    plt.plot(hr[:,0], hr[:,1],linestyle='-', marker='o', label = "HR")
    plt.plot(ndcg[:,0], ndcg[:,1],linestyle='-', marker='v', label = "NDCG")
    plt.plot(loss[:,0], loss[:,1],linestyle='-', marker='s', label = "Loss")

    plt.xlabel("Epochs")
    plt.ylabel("Value")
    plt.legend()    

# --> Testing Init performance
hr, ndcg, mrr = test(model.to(device), ds, testRatings, topK=10)

              Eval: MRR = 0.1509 -- HR = 0.3955 -- NDCG = 0.2075


#### http://www.gabormelli.com/RKB/Mean_Reciprocal_Rank_(MRR)_Measure
MRRs = []

q1 = 3
rank1 = [1,2,3]
MRRs.append(getMRR(rank1, q1))

q2 = 2
rank2 = [1,2,3]
MRRs.append(getMRR(rank2, q2))

q3 = 1
rank3 = [1,2,3]
MRRs.append(getMRR(rank3, q3))

print(MRRs)
>>> [0.3333333333333333, 0.5, 1.0]

np.array(MRRs).mean()
>>> 0.611111111111111



In [ ]:
class RecSysNet(torch.nn.Module):
    def __init__(self, n_users, n_items, embedding_dim=16, out=1):
        super().__init__()

        self.activation = torch.nn.ReLU()
        self.drop = torch.nn.Dropout()
        
        embedding_dim = embedding_dim

        self.user_embedding = torch.nn.Embedding(n_users, embedding_dim//2)
        self.item_embedding = torch.nn.Embedding(n_items, embedding_dim//2)

        self.fc_layer_0 = torch.nn.Linear(embedding_dim, 2 * embedding_dim)

        self.fc_layer_1 = torch.nn.Linear(2 * embedding_dim, embedding_dim//2)

        # self.fc_layer_2 = torch.nn.Linear(embedding_dim, embedding_dim//2)

        self.output_layer = torch.nn.Linear(embedding_dim//2, out)

    def forward(self, feed_dict):
        user_embedding = self.user_embedding(feed_dict['user_id'])
        item_embedding = self.item_embedding(feed_dict['item_id'])

        x = torch.cat([user_embedding, item_embedding], dim=1)

        # o = self.drop(self.activation(self.fc_layer_0(x)))

        o = self.activation(self.fc_layer_0(x))
        o = self.activation(self.fc_layer_1(o))
        # o = self.activation(self.fc_layer_2(o))
        
        logit = self.output_layer(o)
        rating = torch.sigmoid(logit)
        return rating

    def predict(self, feed_dict):
        for key in feed_dict:
            if type(feed_dict[key]) != type(None):
                feed_dict[key] = torch.from_numpy(feed_dict[key]).to(dtype=torch.long, device=device)
        output_scores = self.forward(feed_dict)
        return output_scores.cpu().detach().numpy()

# --> Testing the Network
try:
    del model
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

model = RecSysNet(
    n_users=num_users, 
    n_items=num_items, 
)    

with torch.no_grad():
    out = model(dict_loader)
print(f'out forward shape: {out.shape}', end= '  --  ')
print(f'out sample: {out[0]}')




#========================================================================================
deterministic()

# --> Testing the Network
try:
    del model
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

model = RecSysNet(
    n_users=num_users, 
    n_items=num_items,
    embedding_dim=128 
).to(device)    

loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters())

TOPK = 10
N_EPOCHS = 20

BCE_loss_list, hr_list, ndcg_list, mrr_list = [], [], [], []

# Check Init performance
hr, ndcg, mrr = test(model, full_dataset, testRatings, topK=TOPK)
print()
hr_list.append(hr)
ndcg_list.append(ndcg)
mrr_list.append(mrr)
BCE_loss_list.append(1)


training_stats = []
for epoch in range(1, N_EPOCHS+1):
    loss_train = train_one_epoch(model, train_loader, loss_fn, optimizer, epoch, N_EPOCHS, device)
    hr, ndcg, mrr = test(model,full_dataset, testRatings, topK=TOPK)
    mrr_list.append(mrr)
    hr_list.append(hr)
    ndcg_list.append(ndcg)
    BCE_loss_list.append(loss_train)

    training_stats.append(
        {
            'epoch': epoch, 
            'Training Loss': loss_train, 
            'MRR': mrr, 
            'HR': hr,
            'NDCG':ndcg,
        }
            )

# plot_statistics(hr_list, ndcg_list, BCE_loss_list,model.get_alias(), "./figs")

best_iter = np.argmax(np.array(mrr_list))
best_mrr = mrr_list[best_iter]
best_hr = hr_list[best_iter]
best_ndcg = ndcg_list[best_iter]

# Epoch = 10
# Train Loss: 0.323103608017744
# Eval: HR = 0.4899, NDCG = 0.2761 [0.9 s]

print(f"End. Best Iteration {best_iter}. MRR: {best_mrr :.4} -- HR: {best_hr :.4} -- NDCG: {best_ndcg :.4}")

out forward shape: torch.Size([256, 1])  --  out sample: tensor([0.4884])
Experimento deterministico, seed: 2357 -- Existe 1 GPU Tesla P100-PCIE-16GB disponível.
              Eval: MRR = 0.03428 -- HR = 0.1039 -- NDCG = 0.05004

Epoch [1/20]: Train Loss: 0.3901
              Eval: MRR = 0.1626 -- HR = 0.3955 -- NDCG = 0.2171
Epoch [2/20]: Train Loss: 0.354
              Eval: MRR = 0.1638 -- HR = 0.3977 -- NDCG = 0.2182
Epoch [3/20]: Train Loss: 0.3451
              Eval: MRR = 0.1736 -- HR = 0.4274 -- NDCG = 0.2323
Epoch [4/20]: Train Loss: 0.3298
              Eval: MRR = 0.1916 -- HR = 0.4836 -- NDCG = 0.2594
Epoch [5/20]: Train Loss: 0.3055
              Eval: MRR = 0.2227 -- HR = 0.5239 -- NDCG = 0.2928
Epoch [6/20]: Train Loss: 0.2863
              Eval: MRR = 0.2229 -- HR = 0.5249 -- NDCG = 0.2936
Epoch [7/20]: Train Loss: 0.2721
              Eval: MRR = 0.2344 -- HR = 0.5376 -- NDCG = 0.3056
Epoch [8/20]: Train Loss: 0.2605
              Eval: MRR = 0.2374 -- HR = 0.5461 -- N

In [ ]:
df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')
pd.set_option('precision', 2)
df_stats

#Item Popularity Model

In [ ]:
# Description: Implements the item popularity model for recommendations

# Python imports
import argparse
from time import time
import numpy as np
import scipy.sparse as sp


def parse_args():
    parser = argparse.ArgumentParser(description="Run ItemPop")
    parser.add_argument('--path', nargs='?', default='Data/',
                        help='Input data path.')
    parser.add_argument('--dataset', nargs='?', default='movielens',
                        help='Choose a dataset.')
    parser.add_argument('--num_neg_test', type=int, default=100,
                        help='Number of negative instances to pair with a positive instance while testing')
    
    return parser.parse_args(args=[])


class ItemPop():
    def __init__(self, train_interaction_matrix: sp.dok_matrix):
        """
        Simple popularity based recommender system
        """
        self.__alias__ = "Item Popularity without metadata"
        # Sum the occurences of each item to get is popularity, convert to array and 
        # lose the extra dimension
        self.item_ratings = np.array(train_interaction_matrix.sum(axis=0, dtype=int)).flatten()

    def forward(self):
        pass

    def predict(self, feeddict) -> np.array:
        # returns the prediction score for each (user,item) pair in the input
        items = feeddict['item_id']
        output_scores = [self.item_ratings[itemid] for itemid in items]
        return np.array(output_scores)

    def get_alias(self):
        return self.__alias__

def main():
    args = parse_args()
    path = args.path
    dataset = args.dataset
    num_negatives_test = args.num_neg_test
    print("Model arguments: %s " %(args))

    topK = 10

    # Load data

    t1 = time()
    full_dataset = MovieLensDataset(path + dataset, num_negatives_test= num_negatives_test)
    train, testRatings, testNegatives = full_dataset.trainMatrix, full_dataset.testRatings, full_dataset.testNegatives
    num_users, num_items = train.shape
    print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d"
          % (time()-t1, num_users, num_items, train.nnz, len(testRatings)))

    model = ItemPop(train)
    test(model, full_dataset, topK)

if __name__ == "__main__":
    main()